# TODO:

- Continue moving the cipher functions into python files inside the folder `cipher`.
- Alongside each script `<name>.py` have a corresponding test script `test_<name>.py` with the print statements you made in the notebooks. If you don't have any then create them. If you write new functions try to come up with some. Sometimes you may need more than one statement to test that the function is really doing what it should be.
- I have started you off just so you can see the format I think is best for now
- Also note I've created a `freq_analysis` file where you can put the ngrams and index of coincidence stuff.
- I've created a `main.ipynb` which for now will be where we use the `cipher` package. 
- In this notebook, try to write some code which will go through the full flow chart we created to crack a cipher. 
- Try to use the hillclimb function I defined in `hillclimb.py`. You'll need to have a neighbours function which gets the neighbours of the current key depending on what type of cipher we think it is.
- Take a look at what I've written below for details on how to pass function to functions and also how this can be used to create decorators 
- I would say defining your own decorators is an intermediate/advanced python skill so don't worry too much about it if it's confusing. It's also fairly specific to python so may not be essential to your understanding of general programming principles.


## Passing objects to functions

It is useful to be able to generalise functions we write so we can reuse them in other settings. One way we can do this is with classes, passing instances of classes to a function. For example, you wrote the MinHeapStack class and the Permutation class and used them in functions. This could be generalised by not just creating an instance of those classes in the function but by passing the instance as an argument:
```python
minheap = MinHeapStack()
best_key = hillclimb(minheap, ciphertext, ...)
```
The function `hillclimb` does not know anything about `minheap` but will probably just expect it to behave in a certain way. As long as the object `minheap` obeys the rules `hillclimb` uses it for, we could give it anything. 

This is really useful in programming because it *decouples* the function `hillclimb` from the class `MinHeapStack`. In this case it doesn't matter much (so don't actually do this in the code) since we won't use anything else in place of this `MinHeapStack` class. In theory, however, we could start to use different things which may behave in different ways under the hood but look the same from the outside (ie. have the same method names).

## Passing functions to functions

The same is true for functions. If all we care about is one function that we want to change, it might make more sense to just pass a single function to another function, rather than a whole instance of a class.

Again, by letting a function take another function as an argument we can *decouple* the first function from the second. As long as we know what arguments the function we've given it takes, we can call it and expect it to give the result we want (as long as we've coded it properly).


In [29]:
# an example
def custom_print(message):
    print("This is a custom print: ", message)

def func(print_func, a, b):
    # do some stuff
    c = a + b
    print_func(c)

# pass the normal print function
func(print, 3, 4)
# pass it our custom version
func(custom_print, 3, 4)

7
This is a custom print:  7


As you can see, this allows us to have a sort of *plugin* structure. We don't care how the passed function does what it does, we just make sure its inputs and outputs are the same.

## Passing function with arguments to another function

In [30]:
def score(val1, val2):
    return sum([val2*v1 for v1 in val1])

def find_multiple_scores(score_func, score_args=()):
    return_vals = [None]*5
    for i in range(5):
        a = list(range(i*10,(i+1)*10))
        return_vals[i] = score_func(a, *score_args)
    return return_vals

find_multiple_scores(score, (4,))

[180, 580, 980, 1380, 1780]

So this worked because we created a tuple of arguments `score_args` (which was just a tuple of length one with `val2=4`).

In the hillclimb implementation I put in `hillclimb.py`, this is the technique I used. As long as we know what arguments `score_func` takes, we can pass them as a tuple.

The `*score_args` just unpacks the tuple and this notation is used quite a bit. I've put a sort of minimal example below

In [31]:
tuple_of_things = ('example', 4, 20.5)
print(*tuple_of_things)

example 4 20.5


Similarly, if you had a dictionary rather than a tuple, you could unpack all the things in the dictionary with a `**`.

This is used when you have key word arguments. You can create a dictionary and unpack it 

In [32]:
dic = {'a_string': 'example', 'num': 4, 'flt': 20.5}

def func(a_string='', num=2, flt=2.3):
    print(a_string, num, flt)

func(**dic)

example 4 20.5


You can then put these ideas together by passing a tuple of unknown arguments `*args` and a group of unkown key word arguments `**kwargs` to a function. These arguments could then be used within that function or, in this case, you can just pass them to the `score_func` if that function knows what to do with them.

In [33]:
def find_multiple_scores(score_func, *args, **kwargs):
    return_vals = [None]*5
    for i in range(5):
        a = list(range(i*10,(i+1)*10))
        return_vals[i] = score_func(a, *args, **kwargs)
    return return_vals

find_multiple_scores(score, 4)

[180, 580, 980, 1380, 1780]

As you can see, this ends up working the same as before. **Be careful about using this**, only use when necessary because while it might seem nice and quick to write, it can lead to confusion. Suddenly you don't know what things are being given to `score_func`, anything could be passed which could cause the function to throw an error. It can also just be hard to see what is actually going on now. 

This is why I used the other method for the hillclimb function.

The reason I've shown you this though is because it's used for decorators...

## Timing Code

the example below is the simplest method but is cumbersome to write every time

In [34]:
from time import sleep, perf_counter

start = perf_counter()
sleep(1)
end = perf_counter()
print(end-start)

1.0151584000004732


When you open files, the primitive way would be like this:

In [35]:
f = open('../._encrypted.txt')
lines = f.readlines()
f.close()

print(lines[1])

1. jr wr wkh dlusruw



A better and cleaner way is like this:

In [36]:
with open('../._encrypted.txt') as f:
    lines = f.readlines()

print(lines[1])

1. jr wr wkh dlusruw



I say better because if something breaks in this `with` block, it will make sure the file closes before python shuts down.

Also because it automatically closes the file at the end, we know how long the file is open for, which improves readability and clarity of the code

We can create our own `with` blocks is using a context manager. Below is an example context manager which is a function. It gets the time at the start, then the `yield` statement tells python to leave the function, go do whatever else it needs to do and then come back once it's done the other stuff inside the block. Once it comes back it finds what the time is now and then prints out the difference.

In [37]:
from contextlib import contextmanager

@contextmanager
def timer():
    start = perf_counter()
    yield
    end = perf_counter()
    print("Time Elapsed: ",end-start)

In [38]:
with timer() as t:
    # stuff can happen here but I'll just tell python to wait for 2 seconds
    sleep(2)

Time Elapsed:  2.004457000000002


Another method (and probably more useful way in this context), is to define a class. This timer class has just two methods. `__enter__` tells python to run this function when you open a `with Timer() as t:` block. You can then do whatever you want in the block and once the block is finished or fails it will run the `__exit__` method. Don't worry about the extra arguments for this method.

The cool thing about this way is that we can store how long the `with` block took and then use it at a later time.

In [39]:
class Timer:
    def __enter__(self):
        self.time = perf_counter()
        return self

    def __exit__(self, *args):
        self.time = perf_counter() - self.time

The two ways I use the Timer class below are doing the same thing but clearly the first looks much nicer and is how it's meant to be used. Hopefully this emphasises how the `with` statement is working.

In [40]:

with Timer() as t:
    sleep(2)

# can do other things here
print(t.time)

new_timer = Timer()
new_timer.__enter__()
sleep(2)
new_timer.__exit__()

print(new_timer.time)



2.001679900000454
2.0075294999996913


Context managers are cool but you probably won't need to write your own very often. It is nice to understand what they are though and why they are useful in certain circumstances. Feel free to use the `Timer` class in your code if you want to quickly time how long a block of code is taking.

## Timing functions

If you just want to time how long a function takes then we can use a decorator instead. 

Decorators are statements we put just before a function which *wraps* a function with some extra functionality.

A decorator is essentially a nice way to redefine a function with this extra functionality without having to write some horrible boilerplate code.

To create a decorator, we take a function as an argument and whatever arguments that function would have taken. We can write these arguments with the `*args, **kwargs` syntax I discussed earlier.

This function which is going to be our decorator then defines a new function which is our old function `func` but with the added functionality to it. This decorator function then returns this new function so we can use it whenever we want.

In [41]:
from functools import wraps

def timeit(func):
    start = perf_counter()
    @wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = perf_counter()
        value = func(*args, **kwargs)
        toc = perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

So just to recap, this `wrapper_timer` is a new function we have defined which takes the same arguments as our previous function `func` and does the same thing. The only difference is that it starts a timer at the beginning and then prints how long the function took at the end.

How do we use this decorator function then? Well a rudimentary way would be like this:

In [42]:
def func1():
    sleep(2)

# we want to time func1 so we use timeit to create a new function which will wrao around func1. 
# we call this new function func1 again.
# this has the result of just adding the timing functionality to our original function
func1 = timeit(func1)

func1()

Elapsed time: 2.0006 seconds


The nicer way which decorators give us is to use the `@` syntax. This does what the previous bit did, it automatically redefines the function so that it does the timing stuff.

In [43]:
@timeit
def func2():
    sleep(1)

func2()


Elapsed time: 1.0079 seconds


Hopefully this makes some sense. If not don't worry about it too much, you can just use `Timer` and `@timeit` if you want without knowing too much about the inner workings. This is the true power of decorators, you can be told *how* to use them without knowing what they actually do behind the scenes.